In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 53.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9841cf7f41f83e363038a30eb59f07f3dae9ecdbe9303d9c3a9ff2b6bff8227d
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('Questions').getOrCreate()

In [4]:
spark

# Q.1
# Read the given CSV file in a Hive table 

---



---





# Perform the following tasks using PySpark
# Q2. Read the data from Hive table as spark dataframe

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from pyspark.sql.dataframe import DataFrame
DataFrame = spark.read.csv('/content/drive/MyDrive/boston.csv',header='true', 
                      inferSchema='true')


In [8]:
DataFrame.show(5)

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|18.0|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|       24.0|
|0.027310001| 0.0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729| 0.0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001| 0.0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999| 0.0|2.

# Q3. Get the correlation between dependent and independent variables

In [9]:
import six
for i in DataFrame.columns:
    if not( isinstance(DataFrame.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to MV for ", i, DataFrame.stat.corr('MV',i))

Correlation to MV for  CRIM -0.3883046116575088
Correlation to MV for  ZN 0.36044534463752903
Correlation to MV for  INDUS -0.48372517128143383
Correlation to MV for  CHAS 0.17526017775291847
Correlation to MV for  NOX -0.4273207763683772
Correlation to MV for  RM 0.695359937127267
Correlation to MV for  AGE -0.37695456714288667
Correlation to MV for  DIS 0.24992873873512172
Correlation to MV for  RAD -0.3816262315669168
Correlation to MV for  TAX -0.46853593528654536
Correlation to MV for  PT -0.5077867038116085
Correlation to MV for  B 0.3334608226834164
Correlation to MV for  LSTAT -0.7376627294671615
Correlation to MV for  MV 1.0


In [10]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')


In [11]:
vhouse_df = vectorAssembler.transform(emp)


In [12]:
vhouse_df =vhouse_df.select(['features', 'MV'])
vhouse_df.show(3)

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
+--------------------+-----------+
only showing top 3 rows



# Q4. Build a linear regression model to predict house price

In [13]:
splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [14]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-0.026865544822585994,0.017451512120922365,-0.02731981648094648,2.1091105273566573,-6.883949490075248,3.9008518591348866,-0.013405734952991,-1.026583296481276,0.0,-0.0022627810393452026,-0.9198092050323107,0.00698809510036543,-0.46767432258956393]
Intercept: 27.99437645653195


[link text](https://)# Q5. Evaluate the Linear Regression model by getting the RMSE and R-squared values

In [15]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 4.960451
r2: 0.714863


In [16]:
train_df.describe().show()

+-------+------------------+
|summary|                MV|
+-------+------------------+
|  count|               342|
|   mean|22.452631587862573|
| stddev| 9.303159452048828|
|    min|       5.599999905|
|    max|              50.0|
+-------+------------------+



In [17]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","MV","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
|30.968019149700876|32.20000076|[0.00906,90.0,2.9...|
| 29.35431414324625|35.40000153|[0.01311,90.0,1.2...|
|16.569773234498186|18.89999962|[0.0136,75.0,4.0,...|
|24.521282572885266|30.10000038|[0.01709,90.0,2.0...|
|30.684767089685707|32.90000153|[0.01778,95.0,1.4...|
+------------------+-----------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.72424


In [18]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 4.71054


In [ ]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 10
objectiveHistory: [0.49999999999999956, 0.4305439922001351, 0.2193416498160873, 0.19708735265922883, 0.16899042095445863, 0.1670963052820008, 0.16679017103300825, 0.16640504462016953, 0.1657181172081439, 0.1656253685283487, 0.16557595882950427]
+--------------------+
|           residuals|
+--------------------+
|  -6.117020072235093|
|  0.8324670627464954|
|  -5.966457469099101|
| 0.27693008442520295|
| -1.2453117381711323|
| -3.1079528878423233|
|  10.240053301987729|
|   9.019943167766186|
|  2.6838907008947857|
|  2.1758964443994984|
|  -1.643821135120806|
|   5.968876832412207|
|-0.05995090426608485|
| -10.046860089714421|
|  -3.461108941869089|
|   3.374821972092562|
|  -4.115334868128937|
|   2.414719889130705|
| -1.1300696413305218|
|  0.4814324060976709|
+--------------------+
only showing top 20 rows



In [19]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","MV","features").show()

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
|30.968019149700876|32.20000076|[0.00906,90.0,2.9...|
| 29.35431414324625|35.40000153|[0.01311,90.0,1.2...|
|16.569773234498186|18.89999962|[0.0136,75.0,4.0,...|
|24.521282572885266|30.10000038|[0.01709,90.0,2.0...|
|30.684767089685707|32.90000153|[0.01778,95.0,1.4...|
|25.204184257178223|23.10000038|[0.0187,85.0,4.15...|
|24.229084121831754|       33.0|[0.019509999,17.5...|
| 39.72609128439927|       50.0|[0.020090001,95.0...|
|31.656351883078557|31.10000038|[0.02187,60.0,2.9...|
|30.621517281153686|34.70000076|[0.02729,0.0,7.07...|
|25.468169769199072|28.70000076|[0.029850001,0.0,...|
|22.202913810060927|20.60000038|[0.033059999,0.0,...|
| 28.57473389417049|24.10000038|[0.034449998,82.5...|
|24.457548419408205|19.39999962|[0.03466,35.0,6.0...|
|30.563526745941118|       28.5|[0.035020001,80.0...|
| 36.77724730353733|45.40000

In [20]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'MV')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="MV", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 4.15132
